In [16]:
# Bernoulli Naive Bayes

import pandas as pd

# 베르누이 나이브 베이즈 라이브러리 임포트
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import BernoulliNB

# 모델 정확도 평가를 위해 임포트
from sklearn.metrics import accuracy_score

In [17]:
# 데이터 획득
email_list = [
                {'email title': 'free game only today', 'spam': True},
                {'email title': 'cheapest flight deal', 'spam': True},
                {'email title': 'limited time offer only today only today', 'spam': True},
                {'email title': 'today meeting schedule', 'spam': False},
                {'email title': 'your flight schedule attached', 'spam': False},
                {'email title': 'your credit card statement', 'spam': False}
             ]
df = pd.DataFrame(email_list)
df

,email title,spam
0,free game only today,True
1,cheapest flight deal,True
2,limited time offer only today only today,True
3,today meeting schedule,False
4,your flight schedule attached,False
5,your credit card statement,False


In [18]:
# 데이터 다듬기
# - 사이킷런의 베르누이 나이브 베이즈 분류기는 숫자만 다루기 때문에 치환 필요
df['label'] = df['spam'].map({True:1, False:0})
df

,email title,spam,label
0,free game only today,True,1
1,cheapest flight deal,True,1
2,limited time offer only today only today,True,1
3,today meeting schedule,False,0
4,your flight schedule attached,False,0
5,your credit card statement,False,0


In [19]:
# 학습에 사용될 데이터와 분류값 나눔
# - 이메일 제목으로 학습하고, 레이블은 label을 사용해 스팸 메일인지 여부 확인
df_x = df["email title"]
df_y = df["label"]

In [20]:
# 베르누이 나이브 베이즈의 입력 데이터는 고정된 크기의 벡터여야 함
# 사이킷런의 Count Vectorizer 를 이용하면 간편하게 특정 데이터 내의 모든 단어를 포함한 고정 길이 벡터 생성 가능
# Count Vectorizer 를 생성할 때 binary=True 를 설정하면 이메일 제목에 특정 단어가 출현할 경우 무조건 1 을,
# 단어가 출현하지 않을 경우에는 0 을 갖도록 설정함

cv = CountVectorizer(binary=True)
x_traincv = cv.fit_transform(df_x)

In [21]:
# 실행결과를 보면, 이메일 제목들에서 총 17개 단어가 발견되어 각 이메일 제목이 17개 크기 벡터로 인코딩 됨
# 이메일 제목에 중복된 단어가 있더라도 중복된 횟수로 표현된 것이 아니라 단순히 1 로 표현된 것을 확인할 수 있음
# ex) limited time offer only today : [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0]
encoded_input = x_traincv.toarray()
encoded_input

array([[0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0],
       [0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0],
       [1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1],
       [0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1]])

In [22]:
# 인코딩된 이메일 제목에 어떤 단어들이 포함되어 있는지 알고 싶은 경우
cv.inverse_transform(encoded_input[0])

[array(['free', 'game', 'only', 'today'], dtype='<U9')]

In [23]:
# 벡터의 17개 인덱스 각각이 무슨 단어를 의미하는지 확인
cv.get_feature_names()

['attached',
 'card',
 'cheapest',
 'credit',
 'deal',
 'flight',
 'free',
 'game',
 'limited',
 'meeting',
 'offer',
 'only',
 'schedule',
 'statement',
 'time',
 'today',
 'your']

In [24]:
# 베르누이 나이브 베이즈 모델 학습하기
# 사이킷런의 베르누이 나이브 베이즈 분류기는 기본적으로 스무딩 지원함
# - 학습 데이터에 없던 단어가 테스트 데이터에 있어도 분류가 잘 진행됨

bnb = BernoulliNB()
y_train = df_y.astype('int')
bnb.fit(x_traincv, y_train)

BernoulliNB()

In [25]:
# 테스트 데이터 다듬기
test_email_list = [
                {'email title': 'free flight offer', 'spam': True},
                {'email title': 'hey traveler free flight deal', 'spam': True},
                {'email title': 'limited free game offer', 'spam': True},
                {'email title': 'today flight schedule', 'spam': False},
                {'email title': 'your credit card attached', 'spam': False},
                {'email title': 'free credit card offer only today', 'spam': False}
             ]

test_df = pd.DataFrame(test_email_list)
test_df['label'] = test_df['spam'].map({True:1, False:0})
test_x = test_df["email title"]
test_y = test_df["label"]
x_testcv = cv.transform(test_x)

In [26]:
predictions = bnb.predict(x_testcv)

In [27]:
accuracy_score(test_y, predictions)

0.8333333333333334